# Netflix

## Importar

In [1]:
# %load basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',200)
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
import time
start = time.time()

In [2]:
liga = '/home/ef/Documents/Diplomado/data/'
df = pd.read_csv(liga + 'netflix_titles_examen.csv')
df.drop_duplicates(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6234 entries, 0 to 6233
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   show_id        6234 non-null   int64 
 1   type           6234 non-null   object
 2   title          6234 non-null   object
 3   director       4265 non-null   object
 4   cast           6234 non-null   object
 5   country        6234 non-null   object
 6   date_added     6234 non-null   object
 7   release_year   6234 non-null   int64 
 8   rating_google  6234 non-null   object
 9   duration       6234 non-null   object
 10  listed_in      6234 non-null   object
 11  description    6234 non-null   object
dtypes: int64(2), object(10)
memory usage: 633.1+ KB


In [3]:
df.shape

(6234, 12)

In [4]:
df.sample(4)

,show_id,type,title,director,cast,country,date_added,release_year,rating_google,duration,listed_in,description
440,80170301,Movie,Kung Fu Yoga,Stanley Tong,"Jackie Chan, Disha Patani, Amyra Dastur, Sonu ...","China, India, Nepal","October 6, 2017",2017,TV-PG,107 min,"Action & Adventure, Comedies, International Mo...",Two mismatched archaeologists battle a team of...
1484,81015100,Movie,Sommore: The Reign Continues,Kevin Layne,Sommore,United States,"November 1, 2018",2015,TV-MA,93 min,Stand-Up Comedy,"Self-proclaimed ""Queen of Comedy"" Sommore stor..."
716,80123743,Movie,A Mighty Team,Thomas Sorriaux,"Gérard Depardieu, Chantal Lauby, Medi Sadoun, ...",France,"October 15, 2016",2016,TV-14,97 min,"Comedies, International Movies, Sports Movies","When a fit of anger leads to a serious injury,..."
1302,80009361,TV Show,The Irregular at Magic High School,NaN,"Yuichi Nakamura, Saori Hayami, Yumi Uchiyama, ...",Japan,"November 15, 2017",2014,NR,1 Season,"Anime Series, International TV Shows, Teen TV ...",Siblings Tatsuya and Miyuki enroll at Magic Hi...


## Análisis exploratorio

### Unión con ratings

In [5]:
rating = pd.read_csv(liga + 'rating_examen_2.csv')
rating.drop_duplicates(inplace = True)
rating.sample(4)
## Wtf con la tabla

,Title,Rating,Genre
5068,Secrets Of Her Majest37y S Secret Service,4ii.7,Thriller
6073,The 81Creative Brain,u5.4,Drama
4184,X129x,7e.1,"Adventure, Drama"
5778,Vict67im Number 8,5.o0,"Action, Adventure, Sci-Fi"


In [6]:
## Creamos iteración para eliminar los dígitos
texto = []
for i in rating['Title']:
    result = ''.join([x for x in i if not x.isdigit()])
    texto.append(result.strip())   
rating['Title'] = texto

## Ahora para eliminar letras
texto = []
for i in rating['Rating']:
    result = ''.join([x for x in i if x.isdigit()])
    texto.append(result)   
rating['Rating'] = texto

rating.sample(3)

,Title,Rating,Genre
5408,Jeremiah Tower The Last Magnificent,45,"Comedy, Drama, Romance"
773,Manje Bistre,66,"Comedy, Drama"
2590,I Am Not Madame Bovary,28,"Mystery, Thriller"


In [7]:
## Para unir las tablas ambas deben estar en mismo formato
def clean_text(text, pattern='[^a-zA-Z]'):
    import re
    import unicodedata
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, ' ', cleaned_text.decode('utf-8'), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().split())
    return cleaned_text

df['limpio'] = [clean_text(x) for x in df['title']]
rating['limpio'] = [clean_text(x) for x in rating['Title']]

display(df[['title', 'limpio']].sample())
rating[['Title', 'limpio']].sample()

,title,limpio
1758,Lupin the 3rd: The Castle of Cagliostro: Speci...,lupin the rd the castle of cagliostro special ...


,Title,limpio
2950,The Game Changers,the game changers


In [8]:
## Se unen tablas
df = df.merge(rating, on = 'limpio', how = 'left').copy()
df[['title', 'limpio', 'Rating', 'Genre']].sample(4)

,title,limpio,Rating,Genre
6914,Secrets of Great British Castles,secrets of great british castles,57,Comedy
7701,Space Jungle,space jungle,46,"Action, Drama, History"
4888,Veronica,veronica,62,Horror
7081,The Chef Show,the chef show,71,"Comedy, Romance"


In [9]:
display(df.columns)

## No es necesaria la primer columna ni el título que trajimos de ratings
df.drop(['show_id', 'Title', 'limpio'], axis = 1, inplace = True)
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating_google', 'duration', 'listed_in', 'description',
       'limpio', 'Title', 'Rating', 'Genre'],
      dtype='object')

Index(['type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating_google', 'duration', 'listed_in', 'description',
       'Rating', 'Genre'],
      dtype='object')

### Ausentes

In [10]:
## Cuántos registros de cada columna son valores ausentes?
df.isnull().sum()

type                0
title               0
director         2380
cast                0
country             0
date_added          0
release_year        0
rating_google       0
duration            0
listed_in           0
description         0
Rating              7
Genre               7
dtype: int64

In [11]:
## Dado el % alto en la variable de director, se reemplaza por una etiqueta
df['director'].replace({np.nan:'No aplica'}, inplace = True)

## Ahora, qué registros no tienen rating desde la tabla ratings?
df[df['Rating'].isnull()]['title']

1271                           Love O2O
5013                               TE3N
6908                           Love O2O
7129                                W1A
7278    Mako Mermaids: An H2O Adventure
7503            H2O: Mermaid Adventures
7582                H2O: Just Add Water
Name: title, dtype: object

In [12]:
## No hay pei, se eliminan
df.dropna(inplace = True)
df.reset_index(drop = True, inplace = True)
df.isnull().sum()

type             0
title            0
director         0
cast             0
country          0
date_added       0
release_year     0
rating_google    0
duration         0
listed_in        0
description      0
Rating           0
Genre            0
dtype: int64

In [13]:
## Las mejores 10 películas en rating, FYI
df[df['type'] == 'TV Show'][['title','Rating']].sort_values(by = 'Rating',
                                                             ascending = False).iloc[:10,:]

,title,Rating
4194,Innocent,90
5480,Chasing Cameron,90
7666,Cocomong,90
4010,Breakout,90
7496,Wynonna Earp,86
7044,Reign,85
3837,Extreme Engagement,85
7430,Pac-Man and the Ghostly Adventures,85
7285,Riverdale,84
6324,Behind Enemy Lines,84


## ETL

### Crear funciones

In [14]:
## Función para limpiar columnas de texto (también quita stopwords)
def comen_limpio(df,col):
        import nltk
        nltk.download('stopwords')
        from nltk.corpus import stopwords
        stop_words = stopwords.words('english')

        texto = []
        for x in [n.split() for n in [clean_text(x) for x in df[col]]]:
            aux = []
            for word in x:
                if word not in stop_words:
                    aux.append(word)
            texto.append(aux)
        df[f'{col}_limpio'] = [' '.join(x) for x in texto]

In [15]:
## Función para cambiar a fecha, la columna tiene formato: January 12, 1995
def fechas(df, col):
    import datetime
    df[col] = [datetime.datetime.strptime(x.strip(),
                                          '%B %d, %Y') for x in df[col]]
    df[f'{col}_year'] = pd.DatetimeIndex(df[col]).year
    df[f'{col}_mes'] = pd.DatetimeIndex(df[col]).month
    df[f'{col}_sem'] = df[col].dt.isocalendar().week.astype(int)
    df[f'{col}_diasem'] = df[col].dt.dayofweek
    

In [16]:
def longitud(df,col):
    df[f'{col}_long'] = df[col].str.len()
    df[f'{col}_n_words'] = df[col].str.split(' ').str.len()
    df[f'{col}_n_letters'] = df[col].map(lambda x:sum(map(str.isalpha, x)))

In [17]:
## Función para contar la cantidad de elementos en un registro 
def cont(df,col):
    df[f'{col}_count'] = [len(str(x).split(',')) for x in df[col]]

In [18]:
## Función para separar las primeras 3 categorías por columna
def sep(df,col,n = 3):
    df = df.join(pd.DataFrame(df[col].str.split(', ',
                                                expand = True).iloc[:,:n].values,
                  columns = [f'{col}_{x}' for x in range(1,n+1)])).fillna('No aplica')
    return df

### Aplicarlas

In [19]:
## Se aplica las funciones y algunas extra
# limpiar texto
aux = []
for col in ['title', 'description']:
    comen_limpio(df,col)
    aux += [col] + [f'{col}_limpio']

## Cambia formato y crea año, mes, sem, diasem
fechas(df,'date_added')

## Mide longitud, palabras y palabras del texto original y limpio
for col in aux:
    longitud(df,col)

## La razón entre limpio y original para cada característica
orig = [x for x in aux if not x.endswith('_limpio')]
tipo = ['long', 'n_words', 'n_letters']
for col in orig:
    for x in tipo:
        df[f'key_{col}_{x}'] = (df[f'{col}_limpio_{x}'] / df[f'{col}_{x}']).fillna(0)

## Desplegar número de directores, actores, etc
for col in ['director', 'cast', 'country', 'listed_in', 'Genre']:
    cont(df,col)
## Donde no hay directores, cambiar el count por 0
df.loc[df['director'] == 'No aplica', 'director_count'] = 0

## El top3 de países y géneros, cada uno en una columna
for col in ['country', 'listed_in', 'Genre']:
    df = sep(df,col)
    
## Cambiar formato de duración (para series multiplicar por mil)
aux = []
for x in range(len(df)):
    if df['type'][x] == 'Movie':
        aux.append(int(df['duration'][x][:-4]))
    else:
        aux.append(int(df['duration'][x][:-7])*1e3) ## Se cambia el rango para no mezclar
df['duration_num'] = aux

[nltk_data] Downloading package stopwords to /home/ef/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ef/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Palabras

In [20]:
## Función para expandir las palabras más repetidas en una columna
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = TfidfVectorizer(ngram_range = (1, 1), 
                     min_df = 10, 
                     max_features = 50)
texto = 'description_limpio'
cv.fit(df[texto])
words = cv.get_feature_names()
df = df.join(pd.DataFrame(data = cv.transform(df[texto]).todense(),
                          columns = words))

In [21]:
df.iloc[0,-50:].sum()

1.7295088375621823

### Outliers

In [21]:
## Función para omitir outliers mediante IQR
def outlier(df, x , p = 0.2):
    var = df[x]
    q1 = var.quantile(p/2) #acota el % indicado, tanto máx como mín
    q3 = var.quantile(1 - p/2)
    iqr = q3 - q1

    df = df[(var.isnull()) | ((var >= q1 - 1.5*iqr) & (var <= q3 + 1.5*iqr))].copy()
    df.reset_index(drop = True, inplace = True)
    return df

In [22]:
num = list(df.describe().columns)
num = [x for x in num if x not in words]
display(df[num].describe([.01,.1,.9,.99]))
dim_antes = len(df)

for col in num:
    df = outlier(df,col, p = 0.15)
    
display('{:.2%}'.format(len(df) / dim_antes))  
df[num].describe([.01,.1,.9,.99])

,release_year,date_added_year,date_added_mes,date_added_sem,date_added_diasem,title_long,title_n_words,title_n_letters,title_limpio_long,title_limpio_n_words,title_limpio_n_letters,description_long,description_n_words,description_n_letters,description_limpio_long,description_limpio_n_words,description_limpio_n_letters,key_title_long,key_title_n_words,key_title_n_letters,key_description_long,key_description_n_words,key_description_n_letters,director_count,cast_count,country_count,listed_in_count,Genre_count,duration_num
count,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000,7820.000
mean,2013.381,2018.014,6.856,27.029,3.078,15.417,2.738,13.071,12.324,2.092,11.232,143.236,23.899,116.810,104.426,14.562,90.864,0.772,0.837,0.808,0.729,0.614,0.777,0.780,7.238,1.224,2.245,2.242,611.673
std,8.607,1.165,3.654,16.015,1.760,10.602,1.822,8.840,9.139,1.284,7.976,12.212,3.043,10.230,12.766,1.929,11.402,0.305,0.238,0.302,0.064,0.082,0.062,0.691,4.514,0.644,0.781,0.791,1161.961
min,1925.000,2008.000,1.000,1.000,0.000,1.000,1.000,0.000,0.000,1.000,0.000,82.000,12.000,61.000,28.000,6.000,23.000,0.000,0.143,0.000,0.341,0.300,0.377,0.000,1.000,1.000,1.000,1.000,3.000
1%,1974.190,2015.000,1.000,1.000,0.000,2.000,1.000,0.000,0.000,1.000,0.000,111.000,17.000,90.000,72.000,10.190,62.000,0.000,0.250,0.000,0.564,0.444,0.613,0.000,1.000,1.000,1.000,1.000,30.000
10%,2006.000,2017.000,2.000,4.000,0.000,5.000,1.000,4.000,2.000,1.000,2.000,130.000,20.000,106.000,90.000,12.000,77.000,0.231,0.500,0.293,0.647,0.519,0.696,0.000,1.000,1.000,1.000,1.000,78.000
50%,2016.000,2018.000,7.000,28.000,3.000,13.000,2.000,11.000,11.000,2.000,10.000,145.000,24.000,118.000,105.000,15.000,91.000,0.881,1.000,1.000,0.731,0.609,0.780,1.000,8.000,1.000,2.000,2.000,113.000
90%,2019.000,2019.000,12.000,48.000,5.000,30.000,5.000,25.000,25.000,4.000,22.000,150.000,27.000,124.000,117.000,17.000,103.000,1.000,1.000,1.000,0.810,0.720,0.855,1.000,12.000,2.000,3.000,3.000,1000.000
99%,2019.000,2020.000,12.000,52.000,6.000,49.000,9.000,41.000,41.000,6.000,36.000,155.000,31.000,130.810,131.810,19.000,116.000,1.000,1.000,1.000,0.869,0.842,0.904,2.000,20.000,4.000,3.000,3.000,6000.000
max,2020.000,2020.000,12.000,53.000,6.000,104.000,17.000,84.000,84.000,11.000,74.000,248.000,48.000,206.000,195.000,29.000,170.000,1.000,4.000,1.000,0.948,1.000,0.974,13.000,50.000,12.000,3.000,3.000,15000.000


'92.85%'

,release_year,date_added_year,date_added_mes,date_added_sem,date_added_diasem,title_long,title_n_words,title_n_letters,title_limpio_long,title_limpio_n_words,title_limpio_n_letters,description_long,description_n_words,description_n_letters,description_limpio_long,description_limpio_n_words,description_limpio_n_letters,key_title_long,key_title_n_words,key_title_n_letters,key_description_long,key_description_n_words,key_description_n_letters,director_count,cast_count,country_count,listed_in_count,Genre_count,duration_num
count,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000,7261.000
mean,2014.159,2018.033,6.853,27.057,3.065,15.331,2.729,12.997,12.226,2.077,11.148,142.429,23.779,116.139,103.757,14.466,90.291,0.769,0.831,0.805,0.729,0.614,0.777,0.757,7.114,1.170,2.251,2.239,507.404
std,6.363,1.093,3.654,16.023,1.760,10.380,1.784,8.674,9.008,1.260,7.868,8.852,2.661,7.640,11.088,1.663,10.068,0.308,0.222,0.305,0.064,0.082,0.062,0.548,4.185,0.456,0.780,0.792,750.122
min,1977.000,2012.000,1.000,1.000,0.000,1.000,1.000,0.000,0.000,1.000,0.000,96.000,12.000,78.000,52.000,8.000,45.000,0.000,0.143,0.000,0.453,0.357,0.484,0.000,1.000,1.000,1.000,1.000,3.000
1%,1986.000,2015.000,1.000,1.000,0.000,2.000,1.000,0.000,0.000,1.000,0.000,111.000,17.000,90.000,72.000,10.000,62.000,0.000,0.250,0.000,0.563,0.444,0.613,0.000,1.000,1.000,1.000,1.000,29.600
10%,2007.000,2017.000,2.000,5.000,0.000,5.000,1.000,4.000,0.000,1.000,0.000,130.000,20.000,106.000,89.000,12.000,77.000,0.000,0.500,0.000,0.647,0.519,0.695,0.000,1.000,1.000,1.000,1.000,78.000
50%,2016.000,2018.000,7.000,28.000,3.000,13.000,2.000,11.000,10.000,2.000,10.000,145.000,24.000,118.000,105.000,14.000,91.000,0.875,1.000,1.000,0.730,0.609,0.780,1.000,8.000,1.000,2.000,2.000,112.000
90%,2019.000,2019.000,12.000,48.000,5.000,30.000,5.000,25.000,25.000,4.000,22.000,150.000,27.000,124.000,117.000,17.000,102.000,1.000,1.000,1.000,0.809,0.720,0.854,1.000,11.000,2.000,3.000,3.000,1000.000
99%,2019.000,2020.000,12.000,52.000,6.000,48.000,8.000,40.000,40.000,6.000,35.000,152.000,29.000,127.000,127.000,18.000,112.000,1.000,1.000,1.000,0.869,0.833,0.904,2.000,19.000,3.000,3.000,3.000,3000.000
max,2020.000,2020.000,12.000,53.000,6.000,73.000,13.000,60.000,61.000,8.000,54.000,181.000,34.000,149.000,151.000,22.000,133.000,1.000,1.500,1.000,0.939,1.000,0.960,2.000,28.000,3.000,3.000,3.000,4000.000


### Categóricas

In [23]:
cat = ['type','rating_google',
       'country_1','country_2','country_3',
       'listed_in_1','listed_in_2','listed_in_3',
       'Genre_1','Genre_2','Genre_3']

## Se despliegan las posibles opciones para cada variable categórica,
# agrupando las categorías con menos de cierto % de frecuencia
adj = [] 
for col in cat:
    aux = []
    aux = df[col].value_counts(1, dropna = False).reset_index()
    global adj
    for i in range(len(aux)):
        if aux.iloc[i,1] < .01:
            adj.append(aux.iloc[i,0])

# adj será la lista de elementos p reemplazar por 'Otros', importante para ETL
df[cat] = df[cat].replace(adj, 'Otros')

## Modelado discreto

In [24]:
df['Rating'] = df['Rating'].astype(int)
## Crear rangos de rating para predecir
df['rango_rating'] , bins = pd.qcut(df['Rating'],
                             q = 3,
                             duplicates = 'drop', 
                             retbins = True)
df['rango_rating'] = df['rango_rating'].astype(str)
df['rango_rating'] = [int(x[-5:][:2]) for x in df['rango_rating']]
df['rango_rating'].value_counts(1)

54   0.342
65   0.335
90   0.323
Name: rango_rating, dtype: float64

In [25]:
X = df[cat + num + words]
y = df['rango_rating'].values.ravel()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9)

### OHE

In [26]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)

## Se obtienen las dummies de cat y se une con num (por eso hereda el index)
X_train = pd.DataFrame(ohe.fit_transform(X_train[cat]),
                       columns = list(ohe.get_feature_names(X[cat].columns)), 
                       index = X_train.index).join(X_train[num + words])

## Sólo transform
X_test = pd.DataFrame(ohe.transform(X_test[cat]),
                      columns = list(ohe.get_feature_names(X[cat].columns)), 
                      index = X_test.index).join(X_test[num + words])

### Modelos

In [27]:
from sklearn.ensemble import RandomForestClassifier 
forest = RandomForestClassifier()

param_forest = {'n_estimators': [x for x in range(100, 1500, 100)],
                'max_features': ['auto', 'sqrt', 'log2'],
                'criterion': ['gini', 'entropy'],
                'class_weight': ['balanced', None],
                'min_samples_split': [x for x in range(2, 50, 2)],
                'min_samples_leaf': [x/100 for x in range(5, 55, 5)]
               }

from sklearn.model_selection import RandomizedSearchCV
search_forest = RandomizedSearchCV(param_distributions = param_forest, 
                            cv = 4, 
                            n_jobs = -1, 
                            scoring = 'accuracy',
                            estimator = forest,
                            verbose = 5,
                            n_iter = 10)

search_forest.fit(X_train,y_train)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   57.4s finished


RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'class_weight': ['balanced', None],
                                        'criterion': ['gini', 'entropy'],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [0.05, 0.1, 0.15,
                                                             0.2, 0.25, 0.3,
                                                             0.35, 0.4, 0.45,
                                                             0.5],
                                        'min_samples_split': [2, 4, 6, 8, 10,
                                                              12, 14, 16, 18,
                                                              20, 22, 24, 26,
                                                              28, 30, 32, 34,
                           

In [28]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()

param_ada={'n_estimators':[50,100,200],
           'learning_rate':[.001,0.01,.1]}

search_ada = RandomizedSearchCV(param_distributions = param_ada, 
                                cv = 4, 
                                n_jobs = -1, 
                                scoring = 'accuracy', 
                                estimator = ada, 
                                verbose = 5,
                                n_iter = 9)

search_ada.fit(X_train,y_train)

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   42.7s finished


RandomizedSearchCV(cv=4, estimator=AdaBoostClassifier(), n_iter=9, n_jobs=-1,
                   param_distributions={'learning_rate': [0.001, 0.01, 0.1],
                                        'n_estimators': [50, 100, 200]},
                   scoring='accuracy', verbose=5)

In [29]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(learning_rate = 0.1, 
                    n_estimators = 190,
                    max_depth = 5, 
                    min_child_weight = 2,
                    objective = 'binary:logistic',
                    subsample = 0.9, 
                    colsample_bytree = 0.8, 
                    seed = 22)

xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=190, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=22, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=22, subsample=0.9,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Voting

In [30]:
from sklearn.ensemble import VotingClassifier
vc = VotingClassifier(estimators = [('Forest', search_forest.best_estimator_), 
                                    ('ADA', search_ada.best_estimator_),
                                    ('XGB', xgb)], 
                      voting = 'soft')

vc.fit(X_train, y_train)

VotingClassifier(estimators=[('Forest',
                              RandomForestClassifier(min_samples_leaf=0.15,
                                                     min_samples_split=44,
                                                     n_estimators=800)),
                             ('ADA',
                              AdaBoostClassifier(learning_rate=0.1,
                                                 n_estimators=200)),
                             ('XGB',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.8, gamma=0,
                                            gpu_id=-1, importance_type='gain',
                                            interacti...onstraints='',
                                            learning_rate=0.1, max_delta_step=0,
            

In [31]:
def score_metrics(X, y, estimator):
    from sklearn.model_selection import cross_val_score
    ls_scores_roc = cross_val_score(estimator = estimator,
                                    X = X, y = y, 
                                    scoring = 'accuracy', 
                                    n_jobs = -1, 
                                    cv = 4)
    print(f'Accuracy media: {np.mean(ls_scores_roc):,.2f}, desviación estándar: {np.std(ls_scores_roc)}')

In [32]:
score_metrics(X = X_train, 
              y = y_train, 
              estimator = vc)

Accuracy media: 0.44, desviación estándar: 0.008360130679601034


In [33]:
## Accuracy por modelo en train y test
scores = pd.DataFrame()
scores['Modelo'] = [x[0] for x in vc.estimators] + ['Voting']
scores['Train'] = [x[1].score(X_train, y_train) for x in vc.estimators] + [vc.score(X_train,
                                                                                    y_train)]
scores['Test'] = [x[1].score(X_test, y_test) for x in vc.estimators] + [vc.score(X_test,
                                                                                    y_test)]
scores

,Modelo,Train,Test
0,Forest,0.372,0.377
1,ADA,0.438,0.413
2,XGB,0.841,0.451
3,Voting,0.832,0.451


In [34]:
## Así explican las mejores 10 variables a la target, del mejor modelo antes de VC
pd.DataFrame(zip(X_train.columns, xgb.feature_importances_),
             columns = ['Variable', 'Relevancia']).sort_values(by = 'Relevancia',
                                                               ascending = False).head(10).reset_index(drop = True)

,Variable,Relevancia
0,Genre_2_Comedy,0.043
1,Genre_1_Horror,0.032
2,title_limpio_long,0.020
3,Genre_1_Thriller,0.020
4,Genre_3_Drama,0.019
5,key_title_n_letters,0.017
6,Genre_2_No aplica,0.017
7,Genre_3_Otros,0.016
8,Genre_2_Crime,0.016
9,Genre_2_Horror,0.015


In [35]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_true = y_test, y_pred = vc.predict(X_test))/len(y_test), 
             index = list(sorted(np.unique(y_test))), 
             columns = list(sorted(np.unique(y_test))))

,54,65,90
54,0.180,0.120,0.077
65,0.080,0.140,0.074
90,0.102,0.096,0.131


## Predict

### Lectura

In [36]:
val = pd.read_csv(liga + 'netflix_titles_examen.csv')
val.drop_duplicates(inplace = True)
rating = pd.read_csv(liga + 'rating_examen_2.csv')
rating.drop_duplicates(inplace = True)

## Creamos iteración para eliminar los dígitos
texto = []
for i in rating['Title']:
    result = ''.join([x for x in i if not x.isdigit()])
    texto.append(result.strip())   
rating['Title'] = texto

## Ahora para eliminar letras
texto = []
for i in rating['Rating']:
    result = ''.join([x for x in i if x.isdigit()])
    texto.append(result)   
rating['Rating'] = texto

val['limpio'] = [clean_text(x) for x in val['title']]
rating['limpio'] = [clean_text(x) for x in rating['Title']]

## Se unen tablas
val = val.merge(rating, on = 'limpio', how = 'left').copy()
## No es necesaria la primer columna ni el título que trajimos de ratings
val.drop(['show_id', 'Title', 'limpio'], axis = 1, inplace = True)

## Dado el % alto en la variable de director, se reemplaza por una etiqueta
val['director'].replace({np.nan:'No aplica'}, inplace = True)

### Funciones

In [37]:
## Se aplica las funciones y algunas extra
# limpiar texto
aux = []
for col in ['title', 'description']:
    comen_limpio(val,col)
    aux += [col] + [f'{col}_limpio']

## Cambia formato y crea año, mes, sem, diasem
fechas(val,'date_added')

## Mide longitud, palabras y palabras del texto original y limpio
for col in aux:
    longitud(val,col)

## La razón entre limpio y original para cada característica
orig = [x for x in aux if not x.endswith('_limpio')]
tipo = ['long', 'n_words', 'n_letters']
for col in orig:
    for x in tipo:
        val[f'key_{col}_{x}'] = (val[f'{col}_limpio_{x}'] / val[f'{col}_{x}']).fillna(0)

## Desplegar número de directores, actores, etc
for col in ['director', 'cast', 'country', 'listed_in', 'Genre']:
    cont(val,col)
## Donde no hay directores, cambiar el count por 0
val.loc[val['director'] == 'No aplica', 'director_count'] = 0

## El top3 de países y géneros, cada uno en una columna
for col in ['country', 'listed_in', 'Genre']:
    val = sep(val,col)
    
## Cambiar formato de duración (para series multiplicar por mil)
aux = []
for x in range(len(val)):
    if val['type'][x] == 'Movie':
        aux.append(int(val['duration'][x][:-4]))
    else:
        aux.append(int(val['duration'][x][:-7])*1e3) ## Se cambia el rango para no mezclar
val['duration_num'] = aux

[nltk_data] Downloading package stopwords to /home/ef/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ef/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
val = val.join(pd.DataFrame(data = cv.transform(val[col]).todense(),
                          columns = words))
val[cat] = val[cat].replace(adj, 'Otros')

In [39]:
X_val = val[cat + num + words]
X_val.sample(3)

,type,rating_google,country_1,country_2,country_3,listed_in_1,listed_in_2,listed_in_3,Genre_1,Genre_2,Genre_3,release_year,date_added_year,date_added_mes,date_added_sem,date_added_diasem,title_long,title_n_words,title_n_letters,title_limpio_long,title_limpio_n_words,title_limpio_n_letters,description_long,description_n_words,description_n_letters,description_limpio_long,description_limpio_n_words,description_limpio_n_letters,key_title_long,key_title_n_words,key_title_n_letters,key_description_long,key_description_n_words,key_description_n_letters,director_count,cast_count,country_count,listed_in_count,Genre_count,duration_num,back,become,city,death,documentary,falls,family,father,find,finds,four,friends,get,girl,group,help,high,home,life,lives,love,man,mother,murder,must,new,old,one,past,save,school,secret,series,son,special,student,take,takes,three,time,town,true,two,wife,woman,women,world,year,years,young
1219,Movie,TV-MA,Otros,No aplica,No aplica,Otros,Otros,International Movies,Otros,Drama,Thriller,2004,2017,10,39,6,22,3,19,16,2,15,150,28,116,92,14,79,0.727,0.667,0.789,0.613,0.500,0.681,1,10,1,3,3,116.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
7043,TV Show,TV-PG,United States,No aplica,No aplica,Otros,Otros,No aplica,Otros,Drama,No aplica,2016,2016,9,36,2,8,1,8,8,1,8,139,26,110,96,15,82,1.000,1.000,1.000,0.691,0.577,0.745,0,6,1,2,2,2000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
6019,TV Show,TV-14,Otros,No aplica,No aplica,International TV Shows,Otros,Otros,Drama,Horror,No aplica,2014,2019,8,32,3,7,1,7,7,1,7,147,27,119,94,14,81,1.000,1.000,1.000,0.639,0.519,0.681,0,8,1,3,2,1000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [40]:
val['Rating'] = val['Rating'].replace('No aplica',60)
val['Rating'] = val['Rating'].astype(int)
val['rango_rating'] = pd.cut(val['Rating'],
                             bins = bins)
val['rango_rating'].value_counts(1)

(16.0, 54.0]   0.342
(54.0, 65.0]   0.332
(65.0, 90.0]   0.326
Name: rango_rating, dtype: float64

### Mejor modelo

In [41]:
## El mejor modelo fue el ensamble en paralelo
scores

,Modelo,Train,Test
0,Forest,0.372,0.377
1,ADA,0.438,0.413
2,XGB,0.841,0.451
3,Voting,0.832,0.451


In [42]:
import pickle

## Guardar modelo
with open('netflix_vc.pkl', 'wb') as file:
    pickle.dump(vc, file)

## Abrir un modelo
with open('netflix_vc.pkl', 'rb') as file:
    modelo = pickle.load(file)

## Listo para usarse
modelo

VotingClassifier(estimators=[('Forest',
                              RandomForestClassifier(min_samples_leaf=0.15,
                                                     min_samples_split=44,
                                                     n_estimators=800)),
                             ('ADA',
                              AdaBoostClassifier(learning_rate=0.1,
                                                 n_estimators=200)),
                             ('XGB',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.8, gamma=0,
                                            gpu_id=-1, importance_type='gain',
                                            interacti...onstraints='',
                                            learning_rate=0.1, max_delta_step=0,
            

In [43]:
## Se aplica el One Hot Encoder para los dummies de variables categóricas
X_val = pd.DataFrame(ohe.fit_transform(X_val[cat]),
                       columns = list(ohe.get_feature_names(X[cat].columns)), 
                       index = X_val.index).join(X_val[num + words])
X_val.sample(3)

,type_Movie,type_TV Show,rating_google_NR,rating_google_Otros,rating_google_PG,rating_google_PG-13,rating_google_R,rating_google_TV-14,rating_google_TV-G,rating_google_TV-MA,rating_google_TV-PG,rating_google_TV-Y,rating_google_TV-Y7,rating_google_TV-Y7-FV,country_1_Otros,"country_1_Poland,",country_1_Somalia,country_1_United States,country_2_Armenia,country_2_East Germany,country_2_Kenya,country_2_Namibia,country_2_No aplica,country_2_Otros,country_2_Syria,country_2_United States,country_2_Vatican City,country_3_Albania,country_3_No aplica,country_3_Otros,country_3_Samoa,country_3_Sudan,country_3_United States,listed_in_1_Action & Adventure,listed_in_1_Anime Series,listed_in_1_British TV Shows,listed_in_1_International Movies,listed_in_1_International TV Shows,listed_in_1_Otros,listed_in_2_International Movies,listed_in_2_International TV Shows,listed_in_2_No aplica,listed_in_2_Otros,listed_in_3_International Movies,listed_in_3_International TV Shows,listed_in_3_No aplica,listed_in_3_Otros,Genre_1_Animation,Genre_1_Comedy,Genre_1_Crime,Genre_1_Drama,Genre_1_Horror,Genre_1_No aplica,Genre_1_Otros,Genre_1_Thriller,Genre_2_Comedy,Genre_2_Crime,Genre_2_Drama,Genre_2_Horror,Genre_2_No aplica,Genre_2_Otros,Genre_2_Thriller,Genre_3_Comedy,Genre_3_Crime,Genre_3_Drama,Genre_3_Horror,Genre_3_No aplica,Genre_3_Otros,Genre_3_Thriller,release_year,date_added_year,date_added_mes,date_added_sem,date_added_diasem,title_long,title_n_words,title_n_letters,title_limpio_long,title_limpio_n_words,title_limpio_n_letters,description_long,description_n_words,description_n_letters,description_limpio_long,description_limpio_n_words,description_limpio_n_letters,key_title_long,key_title_n_words,key_title_n_letters,key_description_long,key_description_n_words,key_description_n_letters,director_count,cast_count,country_count,listed_in_count,Genre_count,duration_num,back,become,city,death,documentary,falls,family,father,find,finds,four,friends,get,girl,group,help,high,home,life,lives,love,man,mother,murder,must,new,old,one,past,save,school,secret,series,son,special,student,take,takes,three,time,town,true,two,wife,woman,women,world,year,years,young
2521,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,2018,2019,3,10,4,9,1,9,9,1,9,151,24,124,116,17,100,1.000,1.000,1.000,0.768,0.708,0.806,0,9,1,1,3,1000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
321,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,2019,2019,9,37,4,34,5,26,23,3,21,145,23,119,104,13,92,0.676,0.600,0.808,0.717,0.565,0.773,1,1,1,1,3,65.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1898,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.00

In [44]:
## Hay columnas en este nuevo set que el modelo nunca ha visto
aux = [x for x in X_val.columns if x not in X_train.columns]
aux

['country_1_Poland,',
 'country_1_Somalia',
 'country_2_Armenia',
 'country_2_East Germany',
 'country_2_Kenya',
 'country_2_Namibia',
 'country_2_Syria',
 'country_2_Vatican City',
 'country_3_Albania',
 'country_3_Samoa',
 'country_3_Sudan',
 'Genre_1_No aplica']

In [45]:
## Dado que son países o películas sin género, se omiten
X_val = X_val[[x for x in X_val.columns if x not in aux]].copy()
[x for x in X_val.columns if x not in X_train.columns]

[]

In [46]:
resultado = pd.DataFrame([x[-5:][:2] for x in val['rango_rating'].astype(str)], 
                         columns = ['real']).replace('na','65').astype(int)
resultado['estimado'] = modelo.predict(X_val)

## Buen modelo!
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(confusion_matrix(y_true = resultado['real'],
                                   y_pred = resultado['estimado'])/len(resultado), 
                  index = list(sorted(np.unique(y_test))), 
                  columns = list(sorted(np.unique(y_test))))
display(cm)

## Con buena acertividad (suma de diagonal en la matriz de confusión)
'{:.2%}'.format(np.asarray(cm).trace())

,54,65,90
54,0.263,0.045,0.033
65,0.042,0.261,0.031
90,0.044,0.045,0.237


'76.07%'

In [47]:
## Tono para cuando termina código
from IPython.lib.display import Audio
import numpy as np

framerate = 4410
play_time_seconds = 1

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(5*np.pi*300*t) + np.sin(2*np.pi*240*t)

## La siguiente línea debe ir debajo del código p que suene
Audio(audio_data, rate=framerate, autoplay=True)

In [ ]:
## Tiempo total para correr el notebook
end = time.time()
tiempo_tot = end - start
import math
str(int(math.floor(tiempo_tot/60
                  )
       )
   ) + " minutos con " + '{:.2f}'.format(60*(tiempo_tot/60 - math.floor(tiempo_tot/60
                                                                       )
                                            )
                                        ) + " segundos"